In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import requests

In [ ]:
import os
filedir = 'analysis_crime_seoul'

In [ ]:
filenames = os.listdir(filedir)

In [ ]:
filenames

In [ ]:
file_list = []
for file in filenames:
    if file.endswith('.csv'):
        file_list.append(file)
   
        
file = file_list[-1]

_crime_anal_police = pd.read_csv("{}/{}".format(filedir, file), encoding= 'cp949')
_crime_anal_police.head()

In [ ]:
police_station = _crime_anal_police['구분'].unique()
police_station

In [ ]:
crime_name = _crime_anal_police['죄종'].unique()
crime_name

In [ ]:
crime_class = _crime_anal_police['발생검거'].unique()
crime_class

In [ ]:
result_list = []
for p in police_station:
    temp_dict = {
        '구분': p
    }
    for n in crime_name:
        for c in crime_class:
            key = n+c
            con1 = _crime_anal_police['구분'] == p
            con2 = _crime_anal_police['죄종'] == n
            con3 = _crime_anal_police['발생검거'] == c
            
            value = _crime_anal_police[con1 & con2 & con3]['건수'].values[0]
            temp_dict[key] = value
    result_list.append(temp_dict)
result_list

In [ ]:
crime_anal_police = pd.DataFrame(result_list)
crime_anal_police.head()

In [ ]:
crime_anal_police['구분'] =  crime_anal_police['구분'].apply(lambda x:"서울"+x+"경찰서")

In [ ]:
crime_anal_police

In [ ]:
url = 'https://map.naver.com/v5/api/search'
params = {
    'caller': 'pcweb',
}

from pprint import pprint
import requests

for police in crime_anal_police['구분']:
    params['query'] = police
    resp = requests.get(url, params)
    print(resp)
    pprint(resp.json())
    break

In [ ]:
crim_anal_police

station_address = []
station_lng = []
station_lat = []

url = 'https://map.naver.com/v5/api/search'
params = {
    'caller':'pcweb'
}

for police in crime_anal_police['구분']:
    params['query'] = police
    resp = requests.get(url, params)
    
    data = resp.json()
    geo_info = data['result']['place']['list'][0]
    
    address = geo_info.get('roadAddress')
    
    if not address:
        address = geo_info.get('address')
    
    station_address.append(
        address
    )
    
    station_lng.append(
        geo_ingo.get('x')
    )
    station_lat.append(
        geo.info.get('y')
    )
    

In [ ]:
# instantSearch
url = 'https://map.naver.com/v5/api/instantSearch'
params = {
    'lang':'ko',
    'caller': 'pcweb',
    'types': 'place',
    'coords': '37.563867960512454,127.00276851654054'
    
}

station_address = []
station_lng = []
station_lat = []

for police in crime_anal_police['구분']:
    params['q'] = police
    resp = requests.get(url, params)
    
    station_data = resp.json()
    police_geo = station_data['poi'][0]

    address = police_geo.get('roadAddress', police_geo.get('jibunAddress'))
    station_address.append(address)
    station_lng.append(police_geo.get('x'))
    station_lat.append(police_geo.get('y'))
    
    print(police + "-->" + police_geo.get('roadAddress', address))

In [ ]:
zip(station_address, station_lat, station_lng)

In [ ]:
dict(zip(station_address, station_lat))

In [ ]:
list(zip(station_address, station_lat, station_lng))

In [ ]:
gu_name =[]
for name in station_address:
    tmp = name.split()
    
    tmp_gu = [gu for gu in tmp if gu[-1]=='구'][0]
    
    gu_name.append(tmp_gu)
crime_anal_police['구별'] = gu_name



In [ ]:
crime_anal_police.head()

In [ ]:
crime_anal_police.rename(columns={
    '구분':'관서명'
}, inplace = True) 

In [ ]:
file

In [ ]:
import os

In [ ]:
if not os.path.exists(filedir+'/modified'):
    os.makedirs(filedir+'/modified')

In [ ]:
modified_file = "modified/{}".format(file)

In [ ]:
crime_anal_police.to_csv("{}/{}".format(filedir,modified_file), sep=',',encoding='utf-8')

In [ ]:
crime_anal_police['구별'].value_counts()

In [ ]:
pd.pivot_table?

In [ ]:
df = crime_anal_police.copy()

In [ ]:
df[df['구별']=='중구']

In [ ]:
pd.pivot_table(df, index=['구별'])

In [ ]:
pd.pivot_table(df, index=['구별','관서명'])

In [ ]:
pd.pivot_table(df, index=['구별','관서명'], values=['절도검거', '절도발생'])

In [ ]:
pd.pivot_table(df, index=['구별'], values=['절도검거', '절도발생'], aggfunc=np.max)

In [ ]:
pd.pivot_table(df, index=['구별'], values=['절도검거', '절도발생'], aggfunc=[np.max, np.mean, np.min])
#pd.pivot_table(df, index=['구별'], values=['절도검거', '절도발생'], aggfunc=[np.max, np.mean,np.min])

In [ ]:
pd.pivot_table(df,
              index=['구별'],
              values=['절도검거','절도발생'],
              aggfunc=[np.max, np.mean],
              fill_value=0,
              margins=True,
              margins_name="총계")

In [ ]:
crime_anal_police.loc[crime_anal_police['구별'] == '강남구'

In [ ]:
crime_anal = pd.pivot_table(crime_anal_police, index=['구별'], aggfunc=np.sum)
crime_anal.head()

In [ ]:
crime_name_list = ['강간','강도','살인','절도','폭력']
for crime_name in crime_name_list:
    #검거율 계산.
    temp_series = (crime_anal.loc[:,crime_name+"검거"]/crime_anal.loc[:,crime_name+"발생"])*100
    temp_series[temp_series>100]=100
    
    #검거율 df에 추가
    crime_anal[crime_name+"검거율"] = temp_series
    #검거 데이터 삭제
    crime_anal.drop(crime_name+"검거", axis=1,inplace=True)
    

In [ ]:
crime_anal

In [ ]:
dict(zip([i+"발생" for i in crime_name_list], crime_name_list))

In [ ]:
column_names = dict(zip([i+"발생" for i in crime_name_list], crime_name_list))
column_names

In [ ]:
crime_anal.rename(columns = column_names, inplace=True)
crime_anal.head()

In [ ]:
crime_anal.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
col = ['강간','강도','살인','절도','폭력']

In [ ]:
x = crime_anal[col].values
min_max_scaler = MinMaxScaler()

#변환기는 fit 과 transform 함수 내장. (합친게 fit_transform)
#fit은 변환할 값 저장
#transform은 값 변환
x_scaled = min_max_scaler.fit_transform(x.astype(float))

#오리지널 데이터, 스케일된 데이터
x, x_scaled

In [ ]:
np.min(x), np.max(x)

In [ ]:
crime_anal_norm = pd.DataFrame(x_scaled, columns=col, index=crime_anal.index)
crime_anal_norm

In [ ]:
col2 = [i+"검거율" for i in col]
crime_anal_norm[col2] = crime_anal[col2]
crime_anal_norm.head()

In [ ]:

result_CCTV = pd.read_csv('data/CCTV_result.csv', encoding='uft-8', index_col='구별')
#crime_anal_norm[['인구수', 'CCTV']] = result_CCTV[['인구수','소계']]
#crime_anal_norm.head()

In [ ]:
crime_anal_norm['인구수']=result_CCTV['인구수']
crime_anal_norm['CCTV']=result_CCTV['소계']
crime_anal_norm
